In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
google_data=pd.read_pickle('Google_Alibaba.pkl')
google_data=google_data['maxCPU_usage']
endvalue=google_data.shape[0]
google_data.head()

20244     0.2009
164605    0.1049
308807    0.1226
462097    0.1602
618227    0.1914
Name: maxCPU_usage, dtype: object

In [3]:
tf.VERSION

'1.15.0'

In [4]:
alibaba_data=pd.read_pickle('Alibaba_Google.pkl')
alibaba_data=alibaba_data[0:9654]
alibaba_data.tail()

,0
9649,0.000081
9650,0.000046
9651,0.000047
9652,0.000047
9653,0.000043


In [5]:
google_data=np.array(google_data,dtype=float)
google_data.shape
alibaba_data=np.array(alibaba_data,dtype=float)
alibaba_data=np.ravel(alibaba_data)
alibaba_data.shape

(9654,)

In [6]:
alibaba_data=np.nan_to_num(alibaba_data)
google_data=np.nan_to_num(google_data)

In [7]:
def split(data,step):
    X=[]
    y=[]
    counter=1
    temp=[]
    for i in range(0,len(data)):
        if counter<=3:
            temp.append(data[i])
            counter=counter+1
        else:
            counter=1
            X.append(temp)
            temp=[]
            y.append(data[i])
    return np.array(X),np.array(y)

In [8]:
X_train,y_train=split(alibaba_data,3)
X_test,y_test=split(google_data,3)

In [9]:
model=Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [10]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

In [11]:
model.fit(X_train, y_train, epochs=200, verbose=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [12]:
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [13]:
yhat = model.predict(X_test, verbose=0)

In [14]:
yhat

array([[-0.00785019],
       [-0.01015245],
       [-0.00360883],
       ...,
       [-0.00352687],
       [-0.00330431],
       [-0.00637111]], dtype=float32)

In [19]:
accuracy=np.mean(np.abs(yhat - y_test)/np.abs(y_test))

In [20]:
accuracy

1.0645667573741837

In [18]:
np.save('LSTM_aligoogpred.npy',yhat)